In [2]:
# Import necessary libraries
import pandas
import praw
from dotenv import dotenv_values, load_dotenv
import os
import nltk
from nltk import word_tokenize
import re
import numpy as np
from bs4 import BeautifulSoup

#nltk.download('punkt')
#nltk.download('stopwords')


# Load environment variables from .env file
env = dotenv_values(".env")

# Authenticate with Reddit using PRAW
reddit = praw.Reddit(
    client_id=env["CLIENT_ID"],
    client_secret=env["CLIENT_SECRET"],
    user_agent=env["USER_AGENT"],
    redirect_uri=env["REDIRECT_URI"],
    refresh_token=env["REFRESH_TOKEN"],
)

# Check if the CSV file already exists
csv_file_name = "reddit_posts_with_comments.csv"
if os.path.exists(csv_file_name):
    print("CSV file already exists. Appending new data and avoiding duplicates.")
    df = pandas.read_csv(csv_file_name)  # Read existing CSV into a DataFrame
else:
    print("CSV file does not exist. It will be created after fetching new data.")
    df = pandas.DataFrame(columns=["Title", "Id", "Upvotes", "Comments"])

# Create a subreddit instance
targetObjects = ['conspiracy',
                 'WhitePeopleTwitter', 'politics', 'Republican', 'worldnews']
for subreddit_name in targetObjects:
    subreddit = reddit.subreddit(subreddit_name)

    # Print subreddit name
    print(subreddit.display_name)

    # Lists to store submission information
    titles = []
    scores = []
    ids = []
    comments = []

    # Loop through the newest 21 submissions in the subreddit
    for iteration, submission in enumerate(subreddit.hot(limit=5)):
        print(f"post {iteration}/5")
        # Check if the submission ID already exists in the DataFrame to avoid duplication
        if submission.id not in df["Id"].values:
            # Add submission title to the titles list
            titles.append(submission.title)
            scores.append(submission.score)  # Add upvotes to the scores list
            ids.append(submission.id)  # Add submission ID to the ids list

            # Fetch comments for the current submission
            submission.comments.replace_more(limit=25)
            submission_comments = []
            for comment in submission.comments.list():
                # Check if the comment author's username contains "bot"
                if 'bot' not in comment.name:
                    # Use BeautifulSoup to remove HTML tags from content
                    soup = BeautifulSoup(comment.body, 'lxml')
                    filtered_content = soup.get_text()

                    # Remove URLs from filtered_content
                    filtered_content = re.sub(
                        r'http\S+|www\S+', '', filtered_content)

                    # Remove only #
                    filtered_content = re.sub(r'#', '', filtered_content).lower()
                    submission_comments.append(filtered_content)
            comments.append(submission_comments)

        # Create a DataFrame with the new data
        new_data = pandas.DataFrame(
            {"Title": titles, "Id": ids, "Upvotes": scores, "Comments": comments}
        )

        # Append/concat the new data to the existing DataFrame
        df = pandas.concat([df, new_data], ignore_index=True)

        # Drop duplicates based on the 'Id' column (submission IDs)
        df.drop_duplicates(subset="Id", keep="last", inplace=True)
    # Save the DataFrame to the CSV file
df.to_csv(csv_file_name, index=False)

# Print the shape of the DataFrame and display the first 10 rows
print(df.shape)
print(df.head(10))

print(f"CSV file '{csv_file_name}' has been generated/updated with the new Reddit posts and comments while avoiding duplicates.")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabriele\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabriele\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CSV file does not exist. It will be created after fetching new data.
conspiracy
post 0/5


C:\Users\gabriele\AppData\Local\Temp\ipykernel_13544\950554949.py:69: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(comment.body, 'lxml')
C:\Users\gabriele\AppData\Local\Temp\ipykernel_13544\950554949.py:69: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(comment.body, 'lxml')


post 1/5
post 2/5
post 3/5
post 4/5
WhitePeopleTwitter
post 0/5
post 1/5
post 2/5
post 3/5
post 4/5
politics
post 0/5
post 1/5
post 2/5
post 3/5
post 4/5
Republican
post 0/5
post 1/5
post 2/5
post 3/5
post 4/5
worldnews
post 0/5
post 1/5
post 2/5
post 3/5
post 4/5
(25, 4)
                                               Title       Id Upvotes  \
0                                     What are they?  15l0ipw    1556   
1                                        Based Niger  15kmbz8    4527   
2                The truth is staring us in the face  15liyrq      38   
3  Uh-oh... WEF took a couple of articles down. A...  15l61t8     162   
4  Aaaaand, they’re gone! $500 million gone. It's...  15kul4m     616   
5                        The Alabama Riverboat brawl  15lelpm   11541   
6  It's kinda wild that GOP's main twitter accoun...  15ld1k8    4573   
7                          Why do you think that is?  15lfmot    2684   
8                               Interesting question  15l3ucd   18210 

In [3]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk import word_tokenize

csv_tokenized = "tokenized_csv.csv"
csv_input = "reddit_posts_with_comments.csv"
df = pd.read_csv(csv_input)

# Function to clean the text using regex
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z]', ' ', str(text))
    cleaned_text = cleaned_text.strip()
    cleaned_text = cleaned_text.lower()
    return cleaned_text

# Clean the 'Comments' column
df['Comments'] = df['Comments'].apply(clean_text)

# Tokenize the text data
df['tokenized_text'] = df['Comments'].apply(word_tokenize)

# Removal of stopwords
stopwords_english = set(nltk.corpus.stopwords.words("english"))
df['tokenized_text'] = df['tokenized_text'].apply(lambda tokens: [token for token in tokens if token not in stopwords_english])

# Remove tokens with a single character
df['tokenized_text'] = df['tokenized_text'].apply(lambda tokens: [token for token in tokens if len(token) > 1])

# Drop the unnecessary columns (keep only the 'tokenized_text' column)
df_cleaned = df[['tokenized_text']]

# Save the cleaned DataFrame to the CSV file
df_cleaned.to_csv(csv_tokenized, index=False)

# Print the shape of the DataFrame and display the first 10 rows
print(df_cleaned.shape)
print(df_cleaned.head(10))

print(
    f"CSV file '{csv_tokenized}' has been generated/updated with the tokenized text while avoiding duplicates and cleaning the data."
)


(25, 1)
                                      tokenized_text
0  [meta, sticky, comment, rule, apply, replying,...
1  [meta, sticky, comment, rule, apply, replying,...
2  [meta, sticky, comment, rule, apply, replying,...
3  [meta, sticky, comment, rule, apply, replying,...
4  [meta, sticky, comment, rule, apply, replying,...
5  [one, thing, missing, make, perfect, police, s...
6  [gqp, post, facts, care, wrong, amount, time, ...
7  [much, mystery, black, people, tend, support, ...
8  [mean, candidates, would, platforms, popular, ...
9  [would, guessed, speak, english, crowd, know, ...
CSV file 'tokenized_csv.csv' has been generated/updated with the tokenized text while avoiding duplicates and cleaning the data.


In [ ]:
import pandas as pd
from hatesonar import Sonar

# Load the dataset from the CSV file
csv_file = "reddit_posts_with_comments.csv"
df = pd.read_csv(csv_file)

# Initialize the Sonar hate speech analyzer
sonar = Sonar()

# List to store the results of hate speech analysis for each comment
results = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    comments = eval(row["Comments"])  # Assuming the comments column is a string representation of a list, convert it back to a list using eval()
    
    for comment in comments:
        result = sonar.ping(text=comment)
        results.append(result)

# Optionally, you can add the results to a new DataFrame if you want to analyze them further
results_df = pd.DataFrame(results)

# Print the results DataFrame
print(results_df)

# Optionally, you can save the results DataFrame to a new CSV file
results_csv_file = "results_hate_speech_analysis.csv"
results_df.to_csv(results_csv_file, index=False)


In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from langdetect import detect
from tqdm import tqdm
import concurrent.futures
import requests

# Load the Reddit posts CSV file
input_csv = "reddit_posts_with_comments.csv"
output_csv = "reddit_posts_with_labels.csv"

# Define a mapping of languages to model names
language_to_model = {
    'en': "IMSyPP/hate_speech_en",
    'it': "IMSyPP/hate_speech_it",
    'nl': "IMSyPP/hate_speech_nl",
    'sl': "IMSyPP/hate_speech_slo",
}

# Define the default model for cases where language detection fails
default_model_name = "IMSyPP/hate_speech_en"

# Function to load the appropriate model based on the language
def load_model(language):
    model_name = language_to_model.get(language, default_model_name)

    # Specify the cache directory for local caching
    cache_dir = ".cache"

    # Use a retry mechanism with a timeout
    retries = 3
    timeout = 30  # seconds

    # Retry mechanism to handle timeout issues
    for _ in range(retries):
        try:
            # Load the tokenizer and model
            tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir, timeout=timeout)
            model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir=cache_dir, timeout=timeout)
            return tokenizer, model
        except requests.exceptions.Timeout:
            print("Request timed out, retrying...")
            continue
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    else:
        print(f"Failed to load model '{model_name}' after {retries} retries.")
        return None, None

# Function to analyze a comment and return the results
def analyze_comment(comment, language):
    # Load the appropriate tokenizer and model
    tokenizer, model = load_model(language)

    # Tokenize the comment
    inputs = tokenizer(comment, return_tensors="pt")

    # Model inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the probabilities for each class (non-hate speech, hate speech, offensive speech, violent speech)
    probabilities = torch.softmax(outputs.logits, dim=1).tolist()[0]

    return probabilities, language

# Read the CSV file
df = pd.read_csv(input_csv)

# Total number of comments to process
total_comments = df['Comments'].apply(len).sum()

# Batch size for writing results to CSV
batch_size = 100

# Function to process a batch of comments and store the results in the result lists
def process_batch(comment_list):
    result_comments = []
    result_probabilities_hate = []
    result_probabilities_offensive = []
    result_probabilities_violent = []
    result_languages = []

    for comment in comment_list:
        try:
            # Detect the language of the comment
            language = detect(comment)
        except:
            # Handle language detection errors by using the default model
            language = 'en'

        # Analyze the comment using the appropriate model
        probabilities, language = analyze_comment(comment, language)

        # Store the results
        result_comments.append(comment)
        result_probabilities_hate.append(probabilities[1])
        result_probabilities_offensive.append(probabilities[2])
        result_probabilities_violent.append(probabilities[3])
        result_languages.append(language)

    return result_comments, result_probabilities_hate, result_probabilities_offensive, result_probabilities_violent, result_languages

# Initialize lists to store final results
final_result_comments = []
final_result_probabilities_hate = []
final_result_probabilities_offensive = []
final_result_probabilities_violent = []
final_result_languages = []

# Processed comment count
processed_comments = 0

# Iterate over rows in the CSV
for index, row in tqdm(df.iterrows(), total=len(df)):
    comment_list = row['Comments']

    # Split the comment list into batches
    comment_batches = [comment_list[i:i + batch_size] for i in range(0, len(comment_list), batch_size)]

    # Process batches concurrently
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_batch, batch) for batch in comment_batches]
        for future in concurrent.futures.as_completed(futures):
            # Get the results from each batch
            result_comments, result_probabilities_hate, result_probabilities_offensive, result_probabilities_violent, result_languages = future.result()

            # Append the results to the final lists
            final_result_comments.extend(result_comments)
            final_result_probabilities_hate.extend(result_probabilities_hate)
            final_result_probabilities_offensive.extend(result_probabilities_offensive)
            final_result_probabilities_violent.extend(result_probabilities_violent)
            final_result_languages.extend(result_languages)

            # Update processed comment count
            processed_comments += len(result_comments)

            # Print progress
            print(f"Processed {processed_comments}/{total_comments} comments ({(processed_comments/total_comments)*100:.2f}%)")

# Create a new DataFrame for the final results
result_df = pd.DataFrame({
    'comment': final_result_comments,
    'probabilities_hate': final_result_probabilities_hate,
    'probabilities_offensive': final_result_probabilities_offensive,
    'probabilities_violent': final_result_probabilities_violent,
    'language': final_result_languages
})

# Save the results to a new CSV file
result_df.to_csv(output_csv, index=False)

print("Analysis completed. Results saved to:", output_csv)



C:\Users\gabriele\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\gabriele\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gabriele\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate develo

KeyboardInterrupt: 